In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
import pulp 

# Load data
demand_points = gpd.read_file('data/derived_data/kmeans_clusters.gpkg')
potential_sites = gpd.read_file('data/derived_data/all_pot_sites.gpkg')
durations=pd.read_csv('data/derived_data/distance_matrix_walking.csv')
rcps=gpd.read_file('data/raw_data/geodata_stadt_Zuerich/recycling_sammelstellen/data/stzh.poi_sammelstelle_view.shp')
matrix=pd.read_csv('data/derived_data/distance_matrix_walking.csv')

In [19]:
matrix

,ID,cluster_ID,Walking_Duration_Minutes
0,e_1,0,22.533333
1,e_1,1,65.448333
2,e_1,2,67.503333
3,e_1,3,66.880000
4,e_1,4,49.793333
...,...,...,...
1442995,p_9172,2995,76.830000
1442996,p_9172,2996,93.898333
1442997,p_9172,2997,72.496667
1442998,p_9172,2998,62.645000


In [23]:
#matrix.query('ID=="e_4" and cluster_ID==2521', inplace=True)
#matrix

In [24]:
temp = matrix.copy()
# Extract the numeric part from IDs like e_1, p_55, etc.
temp['numeric_id'] = temp['ID'].str.split('_').str[1].astype(int)
# Sort by that numeric portion
temp.sort_values('numeric_id', inplace=True)

distance_matrix = temp.pivot(index='ID', columns='cluster_ID', values='Walking_Duration_Minutes')
# Extract the numeric part from IDs like e_1, p_55, etc.
temp['numeric_id'] = temp['ID'].str.split('_').str[1].astype(int)
# Sort by that numeric portion
temp.sort_values('numeric_id', inplace=True)

distance_matrix = temp.pivot(index='ID', columns='cluster_ID', values='Walking_Duration_Minutes')

In [26]:
distance_matrix.iloc[3, 2521] # that can't be 89 minutes

86.31333333333333

In [18]:
distance_matrix = matrix.pivot(index='ID', columns='cluster_ID', values='Walking_Duration_Minutes')

In [28]:
distance_matrix

cluster_ID,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
ID,,,,,,,,,,,,,,,,,,,,,
e_1,22.533333,65.448333,67.503333,66.880000,49.793333,63.528333,36.523333,45.870000,95.431667,66.285000,...,51.466667,85.950000,92.676667,38.135000,55.175000,86.948333,5.338333,26.138333,34.170000,22.080000
e_10,49.481667,40.080000,34.503333,62.670000,59.816667,42.840000,22.183333,71.118333,71.663333,83.588333,...,34.768333,73.428333,59.675000,36.281667,55.638333,82.706667,31.276667,8.100000,9.218333,47.930000
e_100,12.620000,64.416667,71.565000,61.715000,42.155000,74.703333,34.110000,48.165000,93.935000,58.210000,...,49.053333,80.975000,96.738333,32.970000,49.543333,81.590000,16.515000,36.120000,36.626667,11.146667
e_101,84.123333,38.658333,81.323333,31.373333,62.081667,108.143333,53.331667,125.581667,37.065000,95.883333,...,39.193333,4.745000,93.931667,47.808333,48.650000,22.131667,87.366667,75.646667,63.741667,82.468333
e_102,97.626667,35.875000,40.460000,78.686667,98.030000,77.858333,61.840000,126.378333,36.643333,128.635000,...,55.875000,57.710000,39.588333,75.055000,90.056667,72.636667,85.278333,63.876667,54.025000,96.171667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
p_9172,106.246667,41.711667,48.983333,83.196667,105.620000,86.478333,70.460000,134.998333,35.221667,137.255000,...,61.710000,60.176667,42.763333,83.675000,97.646667,76.830000,93.898333,72.496667,62.645000,104.791667
p_946,85.658333,46.598333,8.505000,80.035000,88.960000,50.513333,52.385000,110.195000,65.993333,117.828333,...,49.495000,78.533333,23.758333,65.426667,83.203333,93.128333,69.693333,47.178333,41.373333,84.201667
p_955,94.165000,57.418333,15.123333,91.173333,98.450000,49.911667,60.816667,118.603333,75.683333,126.335000,...,60.633333,88.303333,13.436667,74.915000,94.273333,102.900000,78.101667,55.586667,49.881667,92.710000


In [ ]:

# Parameter: number of NEW facilities to open (in addition to already open sites)
p = 12

# Get dimensions from the real distance matrix
num_demand_points = distance_matrix.shape[1]  # number of demand points
num_potential_sites = distance_matrix.shape[0]  # number of potential sites

I = range(num_demand_points)
J = range(num_potential_sites)

# Set population per demand point using total_est_pop from demand_points
pop = demand_points['total_est_pop'].to_dict()

# Identify existing and potential sites
existing = [j for j in J if potential_sites.iloc[j]['status'] == "open"]
new_sites = [j for j in J if potential_sites.iloc[j]['status'] == "potential"]

# Create facility open/closed indicator: fixed at 1 for existing sites,
# and decision variable for potential new sites.
y = {}
for j in J:
    if j in existing:
        y[j] = 1  # existing facility is fixed as open
    else:
        y[j] = pulp.LpVariable(f"Facility_{j}", cat='Binary')

# Create the model
prob = pulp.LpProblem("P-Median_Problem", pulp.LpMinimize)

# Assignment decision variables for all sites (binary: 1 if demand point i is assigned to facility j, else 0)
x = pulp.LpVariable.dicts("Assign", [(i, j) for i in I for j in J], cat='Binary')

# Objective Function: Minimize total weighted distance with distance weighted 4 times more than population
prob += pulp.lpSum((distance_matrix.iloc[j, i]) * x[(i, j)] for i in I for j in J)

# Constraint: Each demand point is assigned to exactly one facility.
for i in I:
    prob += pulp.lpSum(x[(i, j)] for j in J) == 1

# Constraint: A demand point can only be assigned to an open facility.
for i in I:
    for j in J:
        if j in new_sites:
            # For potential sites, deny assignment if facility is not selected.
            prob += x[(i, j)] <= y[j]
        else:
            # For existing sites, they are always open so assignment is allowed.
            prob += x[(i, j)] <= 1

# Constraint: Exactly p new facilities are opened among potential sites.
prob += pulp.lpSum(y[j] for j in new_sites) == p

# Solve the model using Gurobi
prob.solve(pulp.GUROBI(msg=True))

# Results
print(f"Status: {pulp.LpStatus[prob.status]}")
print(f"Total Weighted Distance: {pulp.value(prob.objective)}")

# Get new opened facilities from potential sites where y[j]==1
opened_new = [j for j in new_sites if pulp.value(y[j]) == 1]

# Combine existing sites (already open) with new opened facilities for full set of open facilities.
opened_facilities = existing + opened_new

# Create a dataframe of selected sites
selected_sites = potential_sites.iloc[opened_facilities].copy()

# Construct assignments: a dictionary mapping demand point i to the facility j assigned.
assignments = {(i, j): pulp.value(x[(i, j)]) for i in I for j in J if pulp.value(x[(i, j)]) == 1}

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "EndeavourOS")


INFO:gurobipy:Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "EndeavourOS")


INFO:gurobipy:


CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]


INFO:gurobipy:CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]


...

Optimal solution found (tolerance 1.00e-04)


INFO:gurobipy:Optimal solution found (tolerance 1.00e-04)


Best objective 1.195332166667e+04, best bound 1.195332166667e+04, gap 0.0000%


INFO:gurobipy:Best objective 1.195332166667e+04, best bound 1.195332166667e+04, gap 0.0000%


Gurobi status= 2
Status: Optimal
Total Weighted Distance: 11953.321666666705


In [ ]:
distance_matrix.shape

NameError: name 'distance_matrix' is not defined

In [ ]:
assignments[(2598,3)]

1.0

In [ ]:
# Create a dictionary mapping each facility (j) to a list of demand point indices (i) assigned to it
facility_to_demands = {}
for (i, j) in assignments:
    facility_to_demands.setdefault(j, []).append(i)

# Build a DataFrame with facility index, facility ID and its associated demand points
facility_demand_df = pd.DataFrame([
    {
        'Facility_Index': j, 
        'Facility_ID': potential_sites.iloc[j]['ID'], 
        'Demand_Points': demand_list
    } 
    for j, demand_list in facility_to_demands.items()
])

facility_demand_df.sort_values(by='Facility_Index', inplace=True)
facility_demand_df.head()

,Facility_Index,Facility_ID,Demand_Points
94,0,e_1,"[103, 192, 266, 417, 480, 502, 641, 646, 677, ..."
105,1,e_2,"[118, 233, 460, 612, 670, 740, 819, 1237, 1238..."
63,2,e_3,"[65, 226, 399, 610, 688, 767, 820, 829, 1052, ..."
115,3,e_4,"[138, 309, 383, 404, 469, 659, 1066, 1125, 135..."
137,4,e_5,"[188, 378, 453, 504, 642, 884, 968, 1254, 1270..."


In [ ]:
# Define colors for the first 5 facilities
colors = ['red', 'blue', 'green', 'orange', 'purple']

# Get first 5 facilities from facility_demand_df
first_5_facilities = facility_demand_df.head()

# Create map centered on mean of selected sites
mean_lat = selected_sites_4326.geometry.y.mean()
mean_lon = selected_sites_4326.geometry.x.mean()
m = folium.Map(location=[mean_lat, mean_lon], zoom_start=12)

# For each of first 5 facilities without using iterrows (using itertuples instead)
for k, facility in enumerate(first_5_facilities.itertuples(index=False)):
    fac_color = colors[k]
    
    # Get facility location from selected_sites_4326 using the Facility_Index
    fac_site = selected_sites_4326.iloc[facility.Facility_Index]  # Removed the +1 offset
    fac_coords = [fac_site.geometry.y, fac_site.geometry.x]
    
    # Add facility marker
    folium.CircleMarker(
        location=fac_coords,
        radius=8,
        color=fac_color,
        fill=True,
        fill_color=fac_color,
        popup=f"Facility {fac_site.poi_id}"
    ).add_to(m)
    
    # Get and plot assigned demand points
    demand_pts = demand_points.iloc[facility.Demand_Points]
    demand_pts_4326 = demand_pts.to_crs(epsg=4326)
    
    for demand in demand_pts_4326.itertuples(index=False):
        d_coords = [demand.geometry.y, demand.geometry.x]
        folium.CircleMarker(
            location=d_coords,
            radius=4,
            color=fac_color,
            fill=True,
            fill_color=fac_color,
            popup=f"Pop: {demand.total_est_pop}\nDemand_id {demand.cluster_id}"
        ).add_to(m)

m

In [ ]:
potential_sites.iloc[3]

ID                                   e_4
status                              open
geometry    POINT (2685138.9 1246721.82)
Name: 3, dtype: object

In [267]:

distance_matrix.iloc[[3],[469]]
# this is screwed up, the distance matrix is not correct, just look at the map

cluster_ID,469
ID,
e_101,5.081667


In [270]:

distance_matrix.iloc[3,641]

99.62666666666668

In [94]:
assignments[(1659,230)]

-0.0

In [121]:
# Define a list with 10 distinct colors
colors = ['red', 'blue', 'green', 'orange', 'purple', 'darkred', 'lightblue', 'cadetblue', 'darkgreen', 'pink']

#
facilities = selected_sites.iloc[0:10].copy()
facilities_4326 = facilities.to_crs(epsg=4326)

# Create a new folium map centered on the mean location of these facilities
mean_lat = facilities_4326.geometry.y.mean()
mean_lon = facilities_4326.geometry.x.mean()
m_cluster = folium.Map(location=[mean_lat, mean_lon], zoom_start=12)

# For each facility find its assigned demand points from the assignments dictionary
for idx, (fac_idx, facility) in enumerate(facilities_4326.iterrows()):
    fac_color = colors[idx % len(colors)]
    fac_coords = [facility.geometry.y, facility.geometry.x]
    # Add a facility marker (using a circle marker to better visualize clusters)
    folium.CircleMarker(
        location=fac_coords,
        radius=8,
        color=fac_color,
        fill=True,
        fill_color=fac_color,
        popup=f"Facility {facility['ID']}"
    ).add_to(m_cluster)
    
    # Find all demand point indices assigned to this facility
    assigned_demand_indices = [i for (i, j) in assignments.keys() if j == fac_idx]
    
    # If there are demand points assigned, plot them
    if assigned_demand_indices:
        demand_pts = demand_points.loc[assigned_demand_indices]
        demand_pts_4326 = demand_pts.to_crs(epsg=4326)
        
        for _, demand in demand_pts_4326.iterrows():
            d_coords = [demand.geometry.y, demand.geometry.x]
            folium.CircleMarker(
                location=d_coords,
                radius=4,
                color=fac_color,
                fill=True,
                fill_color=fac_color,
                popup=f"Pop: {demand['total_est_pop'], demand['cluster_id']}"
            ).add_to(m_cluster)

# Display the map
m_cluster


In [123]:
distance_matrix.loc[[10, 2],[2789,1659]]


cluster_ID,2789,1659
10,60.283333,23.693333
2,39.046667,101.906667


In [ ]:
kdemand

cluster_id                                                 103
kmeans_cluster                                             103
total_est_pop                                        89.595614
geometry          POINT (8.564354647000862 47.372071309147074)
Name: 103, dtype: object

In [124]:
import folium

# Convert selected_sites to WGS 84 if not already in lat/lon
selected_sites_4326 = selected_sites.to_crs(epsg=4326)

# Set the map center to the mean coordinates of all sites
mean_lat = selected_sites_4326.geometry.y.mean()
mean_lon = selected_sites_4326.geometry.x.mean()
m = folium.Map(location=[mean_lat, mean_lon], zoom_start=12)

# Add markers with different colors:
# - Open sites (existing) in blue
# - New sites (potential) in red
for idx, row in selected_sites_4326.iterrows():
    coords = [row.geometry.y, row.geometry.x]
    color = 'blue' if row['status'] == 'open' else 'red'
    folium.Marker(location=coords, popup=row['ID'], icon=folium.Icon(color=color)).add_to(m)

m



In [130]:
selected_sites

,ID,status,geometry
0,e_1,open,POINT (2684735.900 1247316.820)
1,e_2,open,POINT (2685595.850 1245890.500)
2,e_3,open,POINT (2684826.510 1246123.150)
3,e_4,open,POINT (2685138.900 1246721.820)
4,e_5,open,POINT (2683164.910 1249970.840)
...,...,...,...
388,p_3084,potential,POINT (2684192.583 1247711.585)
427,p_5054,potential,POINT (2683456.237 1247997.118)
435,p_6011,potential,POINT (2681223.369 1249210.010)
447,p_6650,potential,POINT (2686570.052 1251165.394)


In [125]:
import openrouteservice
import scripts.util as util
import geopandas as gpd

# Load the flats_population dataset (produced by the population allocation step)
flats_population = gpd.read_file('data/derived_data/flats_population.gpkg').to_crs(epsg=4326)

flats_population = flats_population.groupby('egid').agg({'est_pop': 'sum', 'geometry': 'first'}).reset_index()

# Initialize ORS client (using our local ORS instance)
client = openrouteservice.Client(base_url='http://localhost:8080/ors')

selected_sites_4326 = selected_sites.to_crs(epsg=4326)
# Build the BallTree using the selected_sites_4326 as rcp locations
selected_sites_4326 = selected_sites_4326.rename(columns={'ID': 'poi_id'})
tree, rcp_coords, rcp_ids = util.initialize_ball_tree(selected_sites_4326, 'poi_id')

# For each flat in flats_population, find the nearest RCP and its walking duration.
# It is assumed that util.find_nearest_rcp_duration returns a tuple (rcp_id, duration_minutes)
results = flats_population.apply(lambda row: util.find_nearest_rcp_duration(row.geometry, tree, rcp_coords, rcp_ids, client), axis=1)

# Extract durations (in minutes); if no valid duration is returned, assign NaN.
flats_population['duration'] = results.apply(lambda res: res[1] if res is not None and res[1] is not None else np.nan)

# Compute the weighted average walking duration using the 'est_pop' column as weights.
weighted_avg = (flats_population['duration'] * flats_population['est_pop']).sum() / flats_population['est_pop'].sum()

print("Weighted Average Walking Duration (min):", weighted_avg)

INFO:scripts.util:BallTree initialized with RCP coordinates.


Weighted Average Walking Duration (min): 4.074235802674274


In [126]:
# write in min:sec format
minutes = int(weighted_avg)
seconds = int((weighted_avg - minutes) * 60)
print(f"Weighted Average Walking Duration: {minutes} min {seconds} sec")

Weighted Average Walking Duration: 4 min 4 sec


In [ ]:
selected_sites_4326['ID']

KeyError: 'ID'